In [4]:
!python -m pip install scikit-learn==1.5.0

In [5]:
import pickle
import pandas as pd

In [6]:
with open('week4/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [7]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [8]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### What's the standard deviation of the predicted duration for this dataset?

In [10]:
y_pred.std()

np.float64(6.247488852238703)

### 2. Preparing the output

In [11]:
year = 2023
month = 3

In [12]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [13]:
df_result = df[['ride_id']].assign(y_pred=y_pred)

In [14]:
df_result

,ride_id,y_pred
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3403761,2023/03_3403761,11.952635
3403762,2023/03_3403762,20.049958
3403763,2023/03_3403763,11.595336
3403764,2023/03_3403764,13.113178


In [15]:
output_file = 'data/week4_result_file.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

### 3. Convert to script

In [47]:
!jupyter nbconvert homework-week4.ipynb --to script


[NbConvertApp] Converting notebook homework-week4.ipynb to script
[NbConvertApp] Writing 1419 bytes to homework-week4.py


### 4. Hash of sklearn dependency

In [1]:
import json

In [2]:
with open('week4/Pipfile.lock') as f:
    j = json.loads(f.read())
    print (j['default']['scikit-learn']['hashes'][0])

sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c


### 5. Parametize the script

In [2]:
!python week4/homework-week4.py 4 2023

https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet
std deviation= 6.353996941249663
mean =  14.292282936862449


### 6. Docker container

In [17]:
!docker run --platform=linux/amd64 week4-job 05 2023

https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet
std deviation= 1.3881399472264797
mean =  0.19174419265916945
